In [1]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    MixtralModel,
    MixtralConfig,
)
from torch import nn
import collections
import time
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

cuda


In [5]:
layer = 15

double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

config = MixtralConfig(
    num_experts_per_tok=8,
    num_hidden_layers=layer,
)

model = MixtralModel(config).from_pretrained(
    "mistralai/Mixtral-8x7B-v0.1",
    quantization_config=double_quant_config,
    attn_implementation="flash_attention_2",
    config=config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
def load_generated_dataset(
    filename, batch_size=32, dataset_relative_path="../dataset"
):
    dataset = pd.read_csv(f"{dataset_relative_path}/{filename}")
    # print(dataset.head(10))
    dataset_list = dataset["0"].tolist()
    dataloader = torch.utils.data.DataLoader(dataset_list, batch_size=batch_size)
    return dataloader


dataset = load_generated_dataset("pile.csv", batch_size=6)

In [8]:
W3 = 0
ACT_FN = 1

sequence_length = 128
batch_size = 6
num_experts = 8
expert_dims = 14336

layer = 14

experts = model.layers[layer].block_sparse_moe.experts


    

In [12]:
i = 0
for batch in tqdm(dataset):
    if i > 0:
        break
    i += 1

    w3 = torch.zeros(num_experts, sequence_length * batch_size, expert_dims)
    act_fn = torch.zeros(num_experts, sequence_length * batch_size, expert_dims)


    def getActivation(expert_idx, type):
        def hook(model, input, output):
            if type == W3:
                w3[expert_idx] = output.detach()
            elif type == ACT_FN:
                act_fn[expert_idx] = output.detach()

        return hook

    for expert_idx in range(num_experts):
        experts[expert_idx].w3.register_forward_hook(getActivation(expert_idx, W3))
        experts[expert_idx].act_fn.register_forward_hook(getActivation(expert_idx, ACT_FN))

    hooks = []
    for expert_idx in range(num_experts):
        w3_hook = experts[expert_idx].w3.register_forward_hook(getActivation(expert_idx, W3))
        act_fn_hook = experts[expert_idx].act_fn.register_forward_hook(getActivation(expert_idx, ACT_FN))
        hooks.extend([w3_hook, act_fn_hook])

    tokenizer.pad_token = tokenizer.eos_token
    batch_tokens = tokenizer(
        batch,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    output = model(batch_tokens["input_ids"])

    for hook in hooks:
        hook.remove()

    # print(tokenizer.batch_decode(generated_ids))

  0%|          | 0/1939 [00:00<?, ?it/s]


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
for k, v in activations.items():
    print("\n", k, ",",'num_sequences', len(v))
    for i in range(len(v)):
        print(v[i].shape, end="")
        # print(v[i])
        # break

# for expert in range(8):
    


 layer_15_expert_0_act_w1 , num_sequences 4
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])
 expert_0_act_w1 , num_sequences 8
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])
 layer_15_expert_0_w_3 , num_sequences 4
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])
 expert_0_w_3 , num_sequences 8
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])
 layer_15_expert_1_act_w1 , num_sequences 4
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])
 expert_1_act_w1 , num_sequences 8
torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14336])torch.Size([768, 14